## **Productionisation (Model Deserialization)**

In [1]:
# Import Libraries

import numpy as np
from pickle import load

import warnings 
warnings.filterwarnings('ignore')

In [2]:
# Loading pretrained models from pickle file

dicts = load(open('/content/dicts_cols.pkl', 'rb'))
scaler = load(open('/content/scaler.pkl', 'rb'))
xgb_reg = load(open('/content/xgb_model.pkl', 'rb'))
gb_reg = load(open('/content/gbr_model.pkl', 'rb'))
rf_reg = load(open('/content/rf_model.pkl', 'rb'))

In [6]:
# Read the Query Data
print("Enter Laptop Details")

# RAM Type options
ram_type_options = dicts['RAM_Type'].keys()
ram_type = input(f'Enter the RAM Type ({", ".join(ram_type_options)}): ')

# Processor options
processor_options = dicts['Processor'].keys()
processor = input(f'Enter the Processor ({", ".join(processor_options)}): ')

# Storage options
storage_options = dicts['Storage'].keys()
storage = input(f'Enter the Storage ({", ".join(storage_options)}): ')

# OS options
os_options = dicts['OS'].keys()
os = input(f'Enter the Operating System ({", ".join(os_options)}): ')

# Brand options
brand_options = dicts['Brand'].keys()
brand = input(f'Enter the Brand ({", ".join(brand_options)}): ')

ram_size = float(input('Enter the RAM Size: '))
display = float(input('Enter the Display Size: '))

Enter Laptop Details
Enter the RAM Type (DDR4, DDR5, LPDDR3, LPDDR4, LPDDR4X, LPDDR5, Unified Memory): Unified Memory
Enter the Processor (AMD Athlon Dual Core, AMD Ryzen 3, AMD Ryzen 3 Dual Core, AMD Ryzen 3 Hexa Core, AMD Ryzen 3 Quad Core, AMD Ryzen 5, AMD Ryzen 5 Dual Core, AMD Ryzen 5 Hexa Core, AMD Ryzen 5 Quad Core, AMD Ryzen 7 Octa Core, AMD Ryzen 7 Quad Core, AMD Ryzen 9 Octa Core, Intel Celeron Dual Core, Intel Celeron Quad Core, Intel Core i3, Intel Core i5, Intel Core i7, Intel Core i9, Intel Evo Core i5, Intel Pentium Quad Core, Intel Pentium Silver, M1, M1 Max, M1 Pro, M2, Qualcomm Snapdragon 7c Gen 2): M2
Enter the Storage (1 TB HDD, 1 TB HDD, 128 GB SSD, 1 TB HDD, 256 GB SSD, 1 TB HDD, 512 GB SSD, 128 GB SSD, 256 GB SSD, 512 GB SSD): 1 TB HDD, 512 GB SSD
Enter the Operating System (Chrome, DOS, Mac OS, Windows): Mac OS
Enter the Brand (ALIENWARE, APPLE, ASUS, DELL, GIGABYTE, HP, Infinix, Lenovo, MSI, Nokia, RedmiBook, SAMSUNG, Ultimus, Vaio, acer, realme): APPLE
Enter t

In [7]:
# Create the query point
query_point = {
    'RAM_Type': dicts['RAM_Type'][ram_type],
    'Processor': dicts['Processor'][processor],
    'Storage': dicts['Storage'][storage],
    'OS': dicts['OS'][os],
    'Brand': dicts['Brand'][brand],
    'RAM_Size': ram_size,
    'Display': display
}

# Print the query point
print("Query Point:")
print(query_point)

Query Point:
{'RAM_Type': 6, 'Processor': 24, 'Storage': 3, 'OS': 2, 'Brand': 1, 'RAM_Size': 64.0, 'Display': 16.0}


In [8]:
# Define function to preprocess input data
def preprocess_input_data(query_point, dicts_cols, scaler):
   # Scale RAM_Size and Display using the scaler
    query_point['RAM_Size'] = scaler.transform([[query_point['RAM_Size'], 0]])[0][0]
    query_point['Display'] = scaler.transform([[query_point['Display'], 0]])[0][0]
    return query_point

# Apply preprocessing to the query point
query_point_preprocessed = preprocess_input_data(query_point, dicts, scaler)

# Convert the query point to a NumPy array
query_array = np.array(list(query_point_preprocessed.values())).reshape(1, -1)

# Print the query array
print("Query Array:")
print(query_array)

Query Array:
[[ 6.         24.          3.          2.          1.          9.19298246
   0.77192982]]


In [11]:
query_point_transformed = scaler.transform(query_array[:, -2:])
other_features = query_array[:, :-2]  # Select all columns except the last two
query_point_transformed = np.concatenate((other_features, query_point_transformed), axis=1)

query_point_transformed

array([[ 6.        , 24.        ,  3.        ,  2.        ,  1.        ,
        -0.42228378, -0.11528822]])

XGBoost Regressor

In [16]:
# Get the predicted value
y_pred = xgb_reg.predict(query_point_transformed)  

# Back-transform the predicted value 
y_pred_orig = np.exp(y_pred)
y_pred_orig

array([218494.17], dtype=float32)

Gradient Boosting Regressor

In [17]:
# Get the predicted value
y_pred = gb_reg.predict(query_point_transformed)  

# Back-transform the predicted value 
y_pred_orig = np.exp(y_pred)
y_pred_orig

array([211977.33284989])

Random Forest Regressor

In [18]:
# Get the predicted value
y_pred = rf_reg.predict(query_point_transformed)  

# Back-transform the predicted value 
y_pred_orig = np.exp(y_pred)
y_pred_orig

array([255248.56328981])

The Random Forest model is way off. The best model is XGBoost. 